In [ ]:
import tensorflow as tf

from tensorflow.examples.tutorials.mnist import input_data
mnist = input_data.read_data_sets("MNIST_data/", one_hot=True)

In [ ]:
tf.reset_default_graph()

images = tf.placeholder(tf.float32, (None, 784))
labels = tf.placeholder(tf.int64, (None, 10))

weights1 = tf.get_variable("weights1", (784, 100))
layer1 = tf.nn.relu(images @ weights1)

weights2 = tf.get_variable("weights2", (100, 100))
layer2 = tf.nn.relu(layer1 @ weights2)

weights3 = tf.get_variable("weights3", (100, 10))
outputs = layer2 @ weights3

predictions = tf.argmax(outputs, 1)
sparse_labels = tf.argmax(labels, 1)
correct_count_op = tf.reduce_sum(tf.cast(tf.equal(predictions, sparse_labels), tf.int64))

total_count_correct = tf.Variable(0, name="total_count_correct", dtype=tf.int64)
total_count_op = tf.assign_add(total_count_correct, correct_count_op)

loss = tf.losses.softmax_cross_entropy(labels, outputs)

train_op = tf.train.AdamOptimizer().minimize(loss)

total_steps = 100
with tf.Session() as sess:
    sess.run(tf.global_variables_initializer())
    for step in range(total_steps):
        batch_images, batch_labels = mnist.train.next_batch(256)
        _, correct_count = sess.run([train_op, correct_count_op], feed_dict={images: batch_images, labels: batch_labels})
        print("Accuracy (training): {} ({}/256)".format(correct_count/256, correct_count))

    total_steps = 50
    total_correct = 0
    for step in range(total_steps):
        sess.run(total_count_correct.initializer)
        batch_images, batch_labels = mnist.validation.next_batch(100)
        total_correct = total_correct + sess.run(total_count_op, feed_dict={images: batch_images, labels: batch_labels}) 
        
    print("Accuracy (validation): {} ({}/5000)".format(total_correct/5000, total_correct))